In [4]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from xgboost import XGBClassifier #xgboost==0.90

## generate gradients 2 cruise protist trophic mode predictions

In [3]:
#load training data
#from cultures of protists
data = pd.read_csv('Field_training_data.csv')
labels = pd.read_csv('Field_training_labels.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

#drop training data rows that have an unknown trophic mode
idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

#load training features (pfams)
#1046 features
features = pd.read_csv('Extracted_Pfams.csv')

#load transcripts-per-million normalized expression of PFAMs 
#for species across g2 samples
G2_TPM_merged_lats = pd.read_csv('G2_surface_tpm_updatedMarferret_marmicroDb.csv')

In [4]:
#fix PFAM IDs in feature dataset
features = features['Pfam'].str.split('.').str[0]

In [5]:
#drop PFAMs in feature dataset that are not present in any 
#species across G2 samples
pfams_toDrop = ['PF02362', 'PF08017', 'PF04413', 'PF05217', 'PF03986', 'PF08217', 'PF13427', 'PF08802']

In [6]:
#drop PFAMs from feature dataset that are not in G2
#samples
features = features[~features.isin(pfams_toDrop)]

In [7]:
#exclude PFAMs from test dataset that are not in training 
#feature dataset 
G2_TPM_merged_lats = G2_TPM_merged_lats[features]

In [8]:
#exclude PFAMs from training dataset that are not in 
#feature dataset
train_data = train_data[features]

In [9]:
#min max scale training data
X,y = train_data, train_labels['Trophic mode']
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [10]:
#scale g2 test dataset
G2 = scaler.transform(G2_TPM_merged_lats)

In [11]:
#fix xg boost model
model = XGBClassifier(n_estimators=10, learning_rate=0.5, reg_lambda=0.)
model.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.5, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=0.0, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [12]:
#predict trophic mode of protist species across g2 samples
xg_predictions_G2 = model.predict(G2)

In [14]:
#make trophic mode of protist species across g2 samples 
#into dataframe
G2_predictions = pd.DataFrame(data={'xg_pred':xg_predictions_G2})

In [15]:
G2_predictions.to_csv('G2_surface_trophicModePredictions_updatedMarferret_marmicroDb',index=False)

## run ML model with different sets of features

In [16]:
#make dataframes that will have PFAMs used for each 
#bootstrap run, and trophic mode predictions outputed 
#by each bootstrap run
input_pfams = {}
output_predictions ={}

input_pfams = pd.DataFrame(input_pfams)
output_predictions = pd.DataFrame(output_predictions)

In [17]:
#run xgboost model 30 times, each time selecting 90% of 
#the PFAMs, without replacement
for i in range (0, 30):
    features1 = features.sample(frac =.9)
    G2_TPM_merged_lats1 = G2_TPM_merged_lats[features1]
    train_data1 = train_data[features1]
    X,y = train_data1, train_labels['Trophic mode']
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    G2 = scaler.transform(G2_TPM_merged_lats1)
    model = XGBClassifier(n_estimators=10, learning_rate=0.5, reg_lambda=0.)
    model.fit(X,y)
    xg_predictions_G2_1 = model.predict(G2)
    input_pfams[f'col{i}'] = features1
    output_predictions[f'col{i}'] = xg_predictions_G2_1

In [18]:
input_pfams.to_csv('bootstrapPfams.csv')
output_predictions.to_csv('bootstrapPredictions.csv')

In [19]:
#make dataframes that will have number of PFAMs used 
#for each bootstrap run, and trophic mode predictions 
#outputed by each bootstrap run
input_pfamNum = {}
output_predictionAcc ={}

input_pfamNum = pd.DataFrame(input_pfamNum)
output_predictionAcc = pd.DataFrame(output_predictionAcc)

In [20]:
#run xgboost model 1000 times, each time selecting a random 
#proportion of the PFAMs, without replacement
for i in range (0, 1000):
    randFrac = np.random.uniform(low=.001, high=1)
    features1 = features.sample(frac = randFrac)
    G2_TPM_merged_lats1 = G2_TPM_merged_lats[features1]
    train_data1 = train_data[features1]
    X,y = train_data1, train_labels['Trophic mode']
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    G2 = scaler.transform(G2_TPM_merged_lats1)
    model = XGBClassifier(n_estimators=10, learning_rate=0.5, reg_lambda=0.)
    model.fit(X,y)
    xg_predictions_G2_1 = model.predict(G2)
    input_pfamNum[f'col{i}'] = [randFrac]
    output_predictionAcc[f'col{i}'] = xg_predictions_G2_1

In [21]:
input_pfamNum.to_csv('bootstrapNumPfams.csv')
output_predictionAcc.to_csv('bootstrapPredictionsChangeNumPfams.csv')